In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset

In [2]:
import kagglehub
import os
# Download latest version
folder_path = kagglehub.dataset_download("cheedcheed/top1m")

print("Path to dataset files:", folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

if not csv_files:
    raise FileNotFoundError("No CSV files found in the folder!")

# read the first CSV file
file_path = os.path.join(folder_path, csv_files[0])
alexa_top_1m_domain = pd.read_csv(file_path,header=None,names=['rank', 'domain'])
alexa_domains_set = set(alexa_top_1m_domain['domain'].apply(str.lower))

ReadTimeout: HTTPSConnectionPool(host='www.kaggle.com', port=443): Read timed out. (read timeout=5)

In [2]:
# Load the dataset from Hugging Face Hub
train_dataset = load_dataset("kmack/Phishing_urls", split="train")
test_dataset = load_dataset("kmack/Phishing_urls", split="test")
valid_dataset = load_dataset("kmack/Phishing_urls", split="valid")

# Convert to pandas DataFrame
train_df = train_dataset.to_pandas()[:500000:]
test_df = test_dataset.to_pandas()
valid_df = valid_dataset.to_pandas()


all_df = [train_df, test_df,valid_df]
for i, df in enumerate(all_df):
    df['url'] = df['text']
    df.drop('text',axis=1,inplace=True)
print(train_df.head())

   label                                                url
0      0             xenophongroup.com/montjoie/compgns.htm
1      1    www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l
2      1                     guildmusic.edu.au/js/index.htm
3      1  memo.unexpectedrunner.com/ezxgytw4et\nholotili...
4      0  en.wikipedia.org/wiki/Category:American_televi...


In [4]:
#dataset 2
import kagglehub
import os
# Download latest version
folder_path = kagglehub.dataset_download("sid321axn/malicious-urls-dataset")

print("Path to dataset files:", folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

if not csv_files:
    raise FileNotFoundError("No CSV files found in the folder!")

# read the first CSV file
file_path = os.path.join(folder_path, csv_files[0])
df = pd.read_csv(file_path)

df['label'] = df['type'].apply(lambda x: x != 'benign')
df.head()

Path to dataset files: C:\Users\rrpra\.cache\kagglehub\datasets\sid321axn\malicious-urls-dataset\versions\1


,url,type,label
0,br-icloud.com.br,phishing,True
1,mp3raid.com/music/krizz_kaliko.html,benign,False
2,bopsecrets.org/rexroth/cr/1.htm,benign,False
3,http://www.garage-pirenne.be/index.php?option=...,defacement,True
4,http://adventure-nicaragua.net/index.php?optio...,defacement,True


In [3]:
import pandas as pd
total_size = df.shape[0]
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df = df.drop_duplicates()
train_df, test_df, valid_df = df[:int(total_size*0.70):], df[int(total_size*0.70):int(total_size*0.80):], df[int(total_size*0.80)::]

In [4]:
import tldextract
df = pd.concat((train_df, valid_df, test_df))
def extrtact_tld(url):

    ext = tldextract.extract(url)
    # ext = ExtractResult(subdomain, domain, suffix)
    tld = ext.suffix    # "com", "co.uk", "org"
    return tld if tld else 'unknown'

df['tld'] = df['url'].apply(extrtact_tld)
df.head()

,label,url,tld
0,1,www.s.aneonorsunoso.top/,top
1,0,reverbnation.com/lilkillaakacred,com
2,0,biography-center.com/biography_details.php?bid...,com
3,0,aleppocodex.org,org
4,1,www.socketlabs.com/smtp-server/,com


In [5]:
# Count phishing and legitimate occurrences for each TLD
tld_stats = df.groupby(['tld', 'label']).size().unstack(fill_value=0)
tld_stats.columns = ['legit_count', 'phish_count']

# Add totals and phishing rate
tld_stats['total'] = tld_stats['legit_count'] + tld_stats['phish_count']
tld_stats['phish_ratio'] = tld_stats['phish_count'] / tld_stats['total']
tld_stats = tld_stats.sort_values('phish_ratio', ascending=False)

print(tld_stats)

        legit_count  phish_count  total  phish_ratio
tld                                                 
zp.ua             0            1      1          1.0
COM               0            1      1          1.0
Data              0            1      1          1.0
webcam            0            2      2          1.0
wang              0            8      8          1.0
...             ...          ...    ...          ...
edu.rs            1            0      1          0.0
edu.sg            5            0      5          0.0
edu.tr            1            0      1          0.0
abbott            1            0      1          0.0
NET               1            0      1          0.0

[691 rows x 4 columns]


In [6]:
print(type(tld_stats))
import pickle
f= open('tld_encoding_serise_dataset_1.bin','wb')
pickle.dump(tld_stats,file=f)
f.close()

<class 'pandas.core.frame.DataFrame'>


In [2]:
import pandas as pd
import re
import math
from urllib.parse import urlparse
import tldextract
from rapidfuzz import process, fuzz


import pickle
f= open('tld_encoding_serise.bin','rb')
tld_stats = pickle.load(file=f)
f.close()
print(type(tld_stats))

# --- Helper function: Shannon entropy ---
def safe_parse(url: str):
    """Safely parse URLs, adding http:// if missing and handling bad IPv6 parts."""
    if not isinstance(url, str) or not url.strip():
        return urlparse("http://")  

    # Ensure scheme exists
    if not re.match(r'^[a-zA-Z]+://', url):
        url = 'http://' + url

    # Clean invalid brackets that trigger IPv6 errors
    url = re.sub(r'\[.*?\]', '', url)

    try:
        return urlparse(url)
    except ValueError:
        # fallback: strip more aggressively if still malformed
        url = re.sub(r'[^a-zA-Z0-9:/._\-?&=]', '', url)
        return urlparse(url)
def calculate_entropy(string):
    """Measures randomness of characters in the URL."""
    if not string:
        return 0
    freq = {char: string.count(char) for char in set(string)}
    entropy = -sum((count / len(string)) * math.log2(count / len(string)) for count in freq.values())
    return entropy

# --- Main feature extraction function ---
def extract_features(url):
    features = {}
    if not re.match(r'^[hH]+[tT]+[tT]+[pP]+[sS]+://', url):
        url = 'http://' + url
    parsed = safe_parse(url)
    
    # 1️⃣ Basic structural features
    features['url_length'] = len(url)
    features['hostname_length'] = len(parsed.netloc)
    features['path_length'] = len(parsed.path)
    features['num_dots'] = url.count('.')
    features['num_hyphens'] = url.count('-')
    features['num_digits'] = sum(c.isdigit() for c in url)
    features['num_letters'] = sum(c.isalpha() for c in url)
    features['num_params'] = url.count('?')
    features['num_equals'] = url.count('=')
    features['num_slashes'] = url.count('/')
    features['num_at'] = url.count('@')

    # 2️⃣ Lexical / composition cues
    features['has_https'] = 1 if url.lower().startswith('https') else 0
    features['has_ip'] = 1 if re.search(r'(\d{1,3}\.){3}\d{1,3}', parsed.netloc) else 0
    features['has_subdomain'] = 1 if parsed.netloc.count('.') > 1 else 0
    features['has_suspicious_words'] = 1 if re.search(r'(login|secure|verify|update|free|bank|click)', url.lower()) else 0

    # 3️⃣ Domain / TLD features
    extracted = tldextract.extract(url)
    main_domain = f"{extracted.domain}.{extracted.suffix}"
    if ':' in main_domain:  # remove port
        main_domain = main_domain.split(':')[0]
    features['domain_length'] = len(main_domain)
    features['in_alexa_top1m'] = 1 if main_domain in alexa_domains_set else 0
    '''
    if features['in_alexa_top1m'] == 0 and main_domain:  # only check if domain not in top1M
        # find closest match in Alexa domains
        best_match, score, _ = process.extractOne(main_domain, alexa_domains_set, scorer=fuzz.ratio)
        features['closest_alexa_domain'] = best_match
        features['closest_alexa_score'] = score  # 0-100
    else:
        features['closest_alexa_score'] = 1000  # high score to show that it is original url
    '''
    ext = tldextract.extract(url)
    tld = ext.suffix    # "com", "co.uk", "org"
    features['tld'] = tld if tld else 'unknown'
    features['tld_phish_ratio'] = tld_stats['phish_ratio'][features['tld']]  if tld_stats['phish_ratio'][features['tld']]  else 0.5
    features['tld_total_frequency'] = tld_stats['total'][features['tld']] if tld_stats['total'][features['tld']] else 1

    # 4️⃣ Ratios
    features['digit_ratio'] = features['num_digits'] / (features['url_length'] + 1e-5)
    features['special_char_ratio'] = (features['num_hyphens'] + features['num_dots'] + features['num_slashes']) / (features['url_length'] + 1e-5)

    # 5️⃣ Entropy (measures randomness / obfuscation)
    features['url_entropy'] = calculate_entropy(url)

    # 6️⃣ Misplacement indicators
    # '@' symbol used to hide real domain (like "http://evil.com@legit.com")
    features['at_in_domain'] = 1 if '@' in parsed.netloc else 0
    
    # Double slashes '//' appearing after path (used to trick users)
    features['double_slash_in_path'] = 1 if re.search(r'/.+//', parsed.path) else 0

    return features

<class 'pandas.core.frame.DataFrame'>


In [13]:
from bs4 import XMLParsedAsHTMLWarning
import warnings
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)


In [14]:
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import socket

def extract_additional_features(url):
    features = {}
    features['url'] = url # to merger with original dataframe using map
    # --- HTML content (download page) ---
    try:
        response = requests.get(url, timeout=5)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    except:
        html = ''
        soup = None
        return features

    # 1. ServerFormHandler
    forms = soup.find_all('form') if soup else []
    features['ServerFormHandler'] = 1
    for form in forms:
        action = form.get('action')
        if action and (action.startswith('mailto:') or action.strip() == "" or not url.split('/')[2] in action):
            features['ServerFormHandler'] = 0  # suspicious
            break

    # 2. InfoEmail
    features['InfoEmail'] = 1 if re.search(r'mailto:', html) else 0

    # 4. WebsiteForwarding
    try:
        r = requests.get(url, timeout=5, allow_redirects=True)
        features['WebsiteForwarding'] = 1 if len(r.history) <= 2 else 0
    except:
        features['WebsiteForwarding'] = 0


    # 6. DisableRightClick
    features['DisableRightClick'] = 1 if re.search(r'oncontextmenu\s*=\s*"return false"', html, re.IGNORECASE) else 0

    # 7. UsingPopupWindow
    features['UsingPopupWindow'] = 1 if re.search(r'window\.open', html, re.IGNORECASE) else 0

    # 8. IframeRedirection
    features['IframeRedirection'] = 1 if '<iframe' in html.lower() else 0



    # 10. DNSRecording
    try:
        socket.gethostbyname(url.split('/')[2])
        features['DNSRecording'] = 1
    except:
        features['DNSRecording'] = 0


    

    # 14. LinksPointingToPage
    features['LinksPointingToPage'] = len(re.findall(r'<a href=', html)) if html else 0

    return features


In [16]:
from tqdm import tqdm
tqdm.pandas()
df = train_df.loc[:10000,['url']]

df = df.assign(**df.url.progress_apply(lambda url : pd.Series(extract_features(url))))
df.head()

100%|██████████| 10001/10001 [00:02<00:00, 4440.95it/s]


,url,url_length,hostname_length,path_length,num_dots,num_hyphens,num_digits,num_letters,num_params,num_equals,...,domain_length,in_alexa_top1m,tld,tld_phish_ratio,tld_total_frequency,digit_ratio,special_char_ratio,url_entropy,at_in_domain,double_slash_in_path
0,http://37.49.226.178/deusbins/deus.sh4,45,5,33,4,0,11,22,0,0,...,5,0,unknown,0.997217,12217,0.244444,0.222222,4.152638,0,0
1,medical-dictionary.thefreedictionary.com/Galt+...,61,40,14,2,1,0,53,0,0,...,21,1,com,0.223625,398479,0.000000,0.098361,4.152943,0,0
2,www.jscape.com/sshfactory/,33,14,12,2,0,0,26,0,0,...,10,1,com,0.223625,398479,0.000000,0.181818,3.922590,0,0
3,http://www.wsnc.org.au/component/jcalpro/view/983,56,5,44,3,0,3,40,0,0,...,5,0,unknown,0.997217,12217,0.053571,0.196429,4.245142,0,0
4,virtualtourist.com/travel/North_America/Canada...,128,18,103,2,5,7,102,0,0,...,18,1,com,0.223625,398479,0.054687,0.117187,4.722947,0,0


In [15]:

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

urls = df['url'].tolist()
results = []

with ThreadPoolExecutor(max_workers=1000) as executor:  # try 20–50 threads
    futures = {executor.submit(extract_additional_features, url): url for url in urls}
    for future in tqdm(as_completed(futures), total=len(urls), desc="Extracting features"):
        try:
            results.append(future.result())
        except Exception as e:
            results.append({})

Extracting features: 100%|██████████| 70882/70882 [01:11<00:00, 987.69it/s]   


In [16]:
features_df = pd.DataFrame(results)
features_df = features_df.drop_duplicates(subset='url').set_index('url')

df = df.assign(**df.url.progress_apply(lambda url : features_df.loc[url]))
df.head()

100%|██████████| 70882/70882 [00:02<00:00, 32002.39it/s]


,label,url,ServerFormHandler,InfoEmail,WebsiteForwarding,DisableRightClick,UsingPopupWindow,IframeRedirection,DNSRecording,LinksPointingToPage
0,0,absoluteastronomy.com/topics/Sri_Lankan_Navy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,www.angelfire.com/amiga/grotto/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,www.etc-meisai.jp.lmxnzp.shop/kaduxn.php?lia71...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,http://torcache.net/torrent/DCA42EC92836BAC652...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1,shanghaitaihong.cn/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
import pandas as pd

def summarize_dataframe(df):
    summary = pd.DataFrame({
        "DataType": df.dtypes,
        "Non-Null Count": df.count(),
        "Missing Values": df.isnull().sum(),
        "Unique Values": df.nunique(),
        "First Value": df.iloc[0],
        "Sample Values": df.apply(lambda x: x.unique()[:5])
    })
    
    # Optional: Add top 5 value counts per column as a dictionary
    value_counts = {}
    for col in df.columns:
        value_counts[col] = df[col].value_counts().head(5).to_dict()
    
    summary["Top 5 Value Counts"] = pd.Series(value_counts)
    
    return summary

summary_table = summarize_dataframe(df)
print(summary_table)

                     DataType  Non-Null Count  Missing Values  Unique Values  \
url                    object           10001               0           9982   
url_length              int64           10001               0            287   
hostname_length         int64           10001               0             95   
path_length             int64           10001               0            188   
num_dots                int64           10001               0             23   
num_hyphens             int64           10001               0             28   
num_digits              int64           10001               0            118   
num_letters             int64           10001               0            231   
num_params              int64           10001               0              4   
num_equals              int64           10001               0             17   
num_slashes             int64           10001               0             19   
num_at                  int64           

In [ ]:

from tqdm import tqdm
tqdm.pandas()
all_df = [train_df, test_df,valid_df]
for i, df in enumerate(all_df):
    all_df[i] = all_df[i].assign(**df.url.progress_apply(lambda url : pd.Series(extract_features(url))))
train_df, test_df,valid_df = all_df

100%|██████████| 130239/130239 [00:31<00:00, 4117.50it/s]


In [20]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

tqdm.pandas()
all_df =  [train_df, test_df,valid_df] 
for i, df in enumerate(all_df):
    urls = df['url'].tolist()
    results = []

    with ThreadPoolExecutor(max_workers=500) as executor:  # try 20–50 threads
        futures = {executor.submit(extract_additional_features, url): url for url in urls}
        for future in tqdm(as_completed(futures), total=len(urls), desc="Extracting features"):
            try:
                results.append(future.result())
            except Exception as e:
                results.append({})

    features_df = pd.DataFrame(results)
    features_df = features_df.drop_duplicates(subset='url').set_index('url')
    df = df.assign(**df.url.progress_apply(lambda url : features_df.loc[url]))
    all_df[i] = df
valid_df = all_df
train_df, test_df,valid_df = all_df

100%|██████████| 70882/70882 [00:02<00:00, 24581.29it/s]


In [21]:
def summarize_dataframe(df):
    summary = pd.DataFrame({
        "DataType": df.dtypes,
        "Non-Null Count": df.count(),
        "Missing Values": df.isnull().sum(),
        "Unique Values": df.nunique(),
        "First Value": df.iloc[0],
        "Sample Values": df.apply(lambda x: x.unique()[:5])
    })
    
    # Optional: Add top 5 value counts per column as a dictionary
    value_counts = {}
    for col in df.columns:
        value_counts[col] = df[col].value_counts().head(5).to_dict()
    
    summary["Top 5 Value Counts"] = pd.Series(value_counts)
    
    return summary

summary_table = summarize_dataframe(train_df)
print(summary_table)

                     DataType  Non-Null Count  Missing Values  Unique Values  \
url                    object          455833               0         449369   
type                   object          455833               0              4   
label                    bool          455833               0              2   
url_length              int64          455833               0            509   
hostname_length         int64          455833               0            237   
path_length             int64          455833               0            384   
num_dots                int64          455833               0             34   
num_hyphens             int64          455833               0             48   
num_digits              int64          455833               0            208   
num_letters             int64          455833               0            401   
num_params              int64          455833               0             16   
num_equals              int64          4

In [22]:
train_df.to_csv(r'Dataset\second_url_train_df.csv',index=False)
test_df.to_csv(r'Dataset\second_url_test_df.csv',index=False)
valid_df.to_csv(r'Dataset\second_url_validation_df.csv',index=False)